# Cuaderno de pruebas para Entregable 2

Este cuaderno permite testear de forma práctica todas las funcionalidades implementadas en el Entregable 2 del proyecto Flask de gestión de tareas con integración de IA. 

Incluye pruebas de endpoints REST, integración con OpenAI, acumulación de tokens y validación de esquemas, dando continuidad al cuaderno del Entregable 1.

## Arranque automático del backend Flask

Antes de realizar cualquier prueba, este cuaderno puede lanzar automáticamente el servidor Flask en segundo plano si no está ya corriendo. Esto facilita la ejecución de las pruebas sin depender de una terminal externa.

Si ya tienes el backend corriendo, puedes omitir la siguiente celda.

In [1]:
import subprocess
import sys
import time

# Lanza el servidor Flask en segundo plano si no está ya corriendo
try:
    flask_proc = subprocess.Popen([sys.executable, "run.py"])
    print("Servidor Flask lanzado en segundo plano.")
    time.sleep(3)  # Espera a que arranque
except Exception as e:
    print("No se pudo lanzar el servidor Flask:", e)

Servidor Flask lanzado en segundo plano.


### Configuración del entorno y comprobación de dependencias

Esta celda carga las variables de entorno desde el archivo `.env`, verifica la conexión con la API Flask y comprueba que la clave de OpenAI esté disponible. Es fundamental para asegurar que el entorno está correctamente preparado antes de ejecutar las pruebas del cuaderno.

In [2]:
# Configuración del entorno y comprobación de dependencias
import os
import sys
import requests
from dotenv import load_dotenv

# Usar el directorio actual como raíz del proyecto
PROJECT_ROOT = os.getcwd()
ENV_PATH = os.path.join(PROJECT_ROOT, '.env')
load_dotenv(dotenv_path=ENV_PATH, override=True)

# Comprobar conexión con la API (ajusta el puerto si es necesario)
API_URL = os.getenv('API_URL', 'http://127.0.0.1:5000')
try:
    r = requests.get(f'{API_URL}/')
    print('✅ API disponible:', r.status_code)
except Exception as e:
    print('❌ No se pudo conectar con la API:', e)

# Comprobar clave OpenAI
openai_key = os.getenv('OPENAI_API_KEY')
print('OPENAI_API_KEY presente:', bool(openai_key))

✅ API disponible: 404
OPENAI_API_KEY presente: True


## Test de endpoints básicos de gestión de tareas

En esta sección se crearán, listarán, actualizarán y eliminarán tareas usando la API REST del proyecto.

## Creación de tareas básicas para pruebas de IA

En la siguiente celda se crearán 3 tareas típicas del desarrollo de un producto de software, utilizando únicamente los campos básicos requeridos por el sistema (sin descripción detallada, sin categoría y con esfuerzo mínimo). 

Estas tareas servirán como base para que, en las siguientes celdas, la IA las enriquezca automáticamente generando la descripción, la categoría, la estimación de esfuerzo y la auditoría mediante los endpoints inteligentes del backend.

In [8]:
# Crear 3 tareas típicas de desarrollo de un producto de software (description vacío y effort_hours=1)

tareas = [
    {
        'title': 'Diseñar la base de datos',
        'description': 'description',
        'priority': 'alta',
        'effort_hours': 1,  # Se estimará con IA
        'status': 'pendiente',
        'assigned_to': 'Ana'
    },
    {
        'title': 'Implementar autenticación de usuarios',
        'description': 'description',
        'priority': 'media',
        'effort_hours': 1,  # Se estimará con IA
        'status': 'pendiente',
        'assigned_to': 'Luis'
    },
    {
        'title': 'Crear pruebas unitarias',
        'description': 'description',
        'priority': 'baja',
        'effort_hours': 1,  # Se estimará con IA
        'status': 'pendiente',
        'assigned_to': 'Sofía'
    }
]

ids = []
for tarea in tareas:
    resp = requests.post(f'{API_URL}/tasks', json=tarea)
    print(f"Tarea '{tarea['title']}' creada:", resp.status_code, resp.json())
    if resp.status_code == 201:
        ids.append(resp.json().get('id'))

# Mostrar todas las tareas actuales
resp = requests.get(f'{API_URL}/tasks')
print('Tareas actuales:', resp.status_code)
for t in resp.json():
    print(t)

# Los campos de descripción enriquecida, categoría y esfuerzo se completarán usando los endpoints de IA en las siguientes celdas.

Tarea 'Diseñar la base de datos' creada: 201 {'assigned_to': 'Ana', 'category': None, 'description': 'description', 'effort_hours': 1.0, 'id': 1, 'priority': 'alta', 'risk_analysis': None, 'risk_mitigation': None, 'status': 'pendiente', 'title': 'Diseñar la base de datos', 'token_usage': 0}
Tarea 'Implementar autenticación de usuarios' creada: 201 {'assigned_to': 'Luis', 'category': None, 'description': 'description', 'effort_hours': 1.0, 'id': 2, 'priority': 'media', 'risk_analysis': None, 'risk_mitigation': None, 'status': 'pendiente', 'title': 'Implementar autenticación de usuarios', 'token_usage': 0}
Tarea 'Crear pruebas unitarias' creada: 201 {'assigned_to': 'Sofía', 'category': None, 'description': 'description', 'effort_hours': 1.0, 'id': 3, 'priority': 'baja', 'risk_analysis': None, 'risk_mitigation': None, 'status': 'pendiente', 'title': 'Crear pruebas unitarias', 'token_usage': 0}
Tareas actuales: 200
{'assigned_to': 'Ana', 'category': None, 'description': 'description', 'eff

In [9]:
# Solicitar a la IA que añada la descripción a la tarea 2 

if len(ids) >= 2:
    task_id = ids[1]
    resp = requests.post(f'{API_URL}/ai/tasks/describe/{task_id}')
    print(f"Respuesta IA para descripción de la tarea 2 (id={task_id}):", resp.status_code, resp.json())
else:
    print("No se encontró la tarea 2 para solicitar la descripción a la IA.")

Respuesta IA para descripción de la tarea 2 (id=2): 200 {'assigned_to': 'Luis', 'category': None, 'description': '**Descripción de la tarea: Implementar autenticación de usuarios**\n\n1. **Revisión de requisitos**: Analizar los requisitos de autenticación que incluyen el registro de usuarios, inicio de sesión, recuperación de contraseña y verificación de correo electrónico.\n\n2. **Selección de tecnología**: Elegir una biblioteca o framework adecuado para la autenticación (por ejemplo, OAuth, JWT, etc.), asegurándose de que sea compatible con la pila tecnológica del proyecto.\n\n3. **Diseño de base de datos**: \n   - Crear una tabla `usuarios` en la base de datos con los siguientes campos:\n     - `id` (int, PK, autoincremento)\n     - `email` (varchar, único)\n     - `contraseña` (varchar, encriptada)\n     - `fecha_creacion` (datetime)\n     - `estado_verificacion` (booleano)\n\n4. **Registro de usuario**:\n   - Desarrollar una API REST para el registro que:\n     - Reciba `email` y 

In [10]:
# Solicitar a la IA que categorice la tarea 2 

if len(ids) >= 2:
    task_id = ids[1]
    resp = requests.post(f'{API_URL}/ai/tasks/categorize/{task_id}')
    print(f"Respuesta IA para categorización de la tarea 2 (id={task_id}):", resp.status_code, resp.json())
else:
    print("No se encontró la tarea 2 para solicitar la categorización a la IA.")

Respuesta IA para categorización de la tarea 2 (id=2): 200 {'assigned_to': 'Luis', 'category': 'Feature', 'description': '**Descripción de la tarea: Implementar autenticación de usuarios**\n\n1. **Revisión de requisitos**: Analizar los requisitos de autenticación que incluyen el registro de usuarios, inicio de sesión, recuperación de contraseña y verificación de correo electrónico.\n\n2. **Selección de tecnología**: Elegir una biblioteca o framework adecuado para la autenticación (por ejemplo, OAuth, JWT, etc.), asegurándose de que sea compatible con la pila tecnológica del proyecto.\n\n3. **Diseño de base de datos**: \n   - Crear una tabla `usuarios` en la base de datos con los siguientes campos:\n     - `id` (int, PK, autoincremento)\n     - `email` (varchar, único)\n     - `contraseña` (varchar, encriptada)\n     - `fecha_creacion` (datetime)\n     - `estado_verificacion` (booleano)\n\n4. **Registro de usuario**:\n   - Desarrollar una API REST para el registro que:\n     - Reciba `e

In [ ]:
# Solicitar a la IA que estime el esfuerzo de la tarea 2 

if len(ids) >= 2:
    task_id = ids[1]
    resp = requests.post(f'{API_URL}/ai/tasks/estimate/{task_id}')
    print(f"Respuesta IA para estimación de esfuerzo de la tarea 2 (id={task_id}):", resp.status_code, resp.json())
else:
    print("No se encontró la tarea 2 para solicitar la estimación de esfuerzo a la IA.")

Respuesta IA para estimación de esfuerzo de la tarea 2 (id=2): 200 {'assigned_to': 'Luis', 'category': 'Feature', 'description': '**Descripción de la tarea: Implementar autenticación de usuarios**\n\n1. **Revisión de requisitos**: Analizar los requisitos de autenticación que incluyen el registro de usuarios, inicio de sesión, recuperación de contraseña y verificación de correo electrónico.\n\n2. **Selección de tecnología**: Elegir una biblioteca o framework adecuado para la autenticación (por ejemplo, OAuth, JWT, etc.), asegurándose de que sea compatible con la pila tecnológica del proyecto.\n\n3. **Diseño de base de datos**: \n   - Crear una tabla `usuarios` en la base de datos con los siguientes campos:\n     - `id` (int, PK, autoincremento)\n     - `email` (varchar, único)\n     - `contraseña` (varchar, encriptada)\n     - `fecha_creacion` (datetime)\n     - `estado_verificacion` (booleano)\n\n4. **Registro de usuario**:\n   - Desarrollar una API REST para el registro que:\n     - R

In [ ]:
# Solicitar a la IA que audite la tarea 2 

if len(ids) >= 2:
    task_id = ids[1]
    resp = requests.post(f'{API_URL}/ai/tasks/audit/{task_id}')
    print(f"Respuesta IA para auditoría de la tarea 2 (id={task_id}):", resp.status_code, resp.json())
else:
    print("No se encontró la tarea 2 para solicitar la auditoría a la IA.")

Respuesta IA para auditoría de la tarea 2 (id=2): 200 {'assigned_to': 'Luis', 'category': 'Feature', 'description': '**Descripción de la tarea: Implementar autenticación de usuarios**\n\n1. **Revisión de requisitos**: Analizar los requisitos de autenticación que incluyen el registro de usuarios, inicio de sesión, recuperación de contraseña y verificación de correo electrónico.\n\n2. **Selección de tecnología**: Elegir una biblioteca o framework adecuado para la autenticación (por ejemplo, OAuth, JWT, etc.), asegurándose de que sea compatible con la pila tecnológica del proyecto.\n\n3. **Diseño de base de datos**: \n   - Crear una tabla `usuarios` en la base de datos con los siguientes campos:\n     - `id` (int, PK, autoincremento)\n     - `email` (varchar, único)\n     - `contraseña` (varchar, encriptada)\n     - `fecha_creacion` (datetime)\n     - `estado_verificacion` (booleano)\n\n4. **Registro de usuario**:\n   - Desarrollar una API REST para el registro que:\n     - Reciba `email`

In [13]:
# Solicitar a la IA descripción, categoría, estimación de esfuerzo y auditoría para la tarea 3 

if len(ids) >= 3:
    task_id = ids[2]
    # Descripción
    resp_desc = requests.post(f'{API_URL}/ai/tasks/describe/{task_id}')
    print(f"Descripción IA tarea 3 (id={task_id}):", resp_desc.status_code, resp_desc.json())
    # Categoría
    resp_cat = requests.post(f'{API_URL}/ai/tasks/categorize/{task_id}')
    print(f"Categoría IA tarea 3 (id={task_id}):", resp_cat.status_code, resp_cat.json())
    # Estimación de esfuerzo
    resp_est = requests.post(f'{API_URL}/ai/tasks/estimate/{task_id}')
    print(f"Esfuerzo IA tarea 3 (id={task_id}):", resp_est.status_code, resp_est.json())
    # Auditoría
    resp_aud = requests.post(f'{API_URL}/ai/tasks/audit/{task_id}')
    print(f"Auditoría IA tarea 3 (id={task_id}):", resp_aud.status_code, resp_aud.json())
else:
    print("No se encontró la tarea 3 para solicitar los endpoints de IA.")

Descripción IA tarea 3 (id=3): 200 {'assigned_to': 'Sofía', 'category': None, 'description': '**Descripción de la tarea: Crear pruebas unitarias para el módulo de autenticación de usuarios**\n\n1. **Objetivo**: Implementar pruebas unitarias que verifiquen la funcionalidad del módulo de autenticación de usuarios, asegurando que todas las funciones relacionadas al login, registro y validación de JWT se comporten como se espera.\n\n2. **Requisitos**:\n   - Revisar el código actual del módulo de autenticación ubicado en `/src/auth`.\n   - Identificar las funciones clave a probar:\n     - `loginUser(credentials)`: Debe retornar un token JWT válido o un error en caso de credenciales incorrectas.\n     - `registerUser(userData)`: Debe crear un nuevo usuario y retornar los datos del usuario o un error en caso de datos inválidos.\n     - `validateToken(token)`: Debe verificar la validez del token y retornar un booleano o un error.\n\n3. **Herramientas**:\n   - Utilizar Jest como framework de pr

In [14]:
# Extraer y mostrar los tokens consumidos de las 3 tareas creadas

token_usages = []
for i, task_id in enumerate(ids):
    resp = requests.get(f'{API_URL}/tasks/{task_id}')
    if resp.status_code == 200:
        tarea = resp.json()
        tokens = tarea.get('token_usage', None)
        print(f"Tarea {i+1} (id={task_id}) - Tokens consumidos: {tokens}")
        token_usages.append(tokens)
    else:
        print(f"No se pudo obtener la tarea {i+1} (id={task_id})")

token_usages

Tarea 1 (id=1) - Tokens consumidos: 0
Tarea 2 (id=2) - Tokens consumidos: 4582
Tarea 3 (id=3) - Tokens consumidos: 4375


[0, 4582, 4375]

In [16]:
# Mostrar toda la información actual de la tarea 3 (Crear pruebas unitarias)

import json

if len(ids) >= 3:
    task_id = ids[2]
    resp = requests.get(f'{API_URL}/tasks/{task_id}')
    if resp.status_code == 200:
        tarea = resp.json()
        print(f"Información completa de la tarea 3 (id={task_id}):")
        print(json.dumps(tarea, indent=2, ensure_ascii=False))
    else:
        print(f"No se pudo obtener la tarea 3: {resp.status_code}")
else:
    print("No se encontró la tarea 3 para mostrar su información.")

Información completa de la tarea 3 (id=3):
{
  "assigned_to": "Sofía",
  "category": "Testing",
  "description": "**Descripción de la tarea: Crear pruebas unitarias para el módulo de autenticación de usuarios**\n\n1. **Objetivo**: Implementar pruebas unitarias que verifiquen la funcionalidad del módulo de autenticación de usuarios, asegurando que todas las funciones relacionadas al login, registro y validación de JWT se comporten como se espera.\n\n2. **Requisitos**:\n   - Revisar el código actual del módulo de autenticación ubicado en `/src/auth`.\n   - Identificar las funciones clave a probar:\n     - `loginUser(credentials)`: Debe retornar un token JWT válido o un error en caso de credenciales incorrectas.\n     - `registerUser(userData)`: Debe crear un nuevo usuario y retornar los datos del usuario o un error en caso de datos inválidos.\n     - `validateToken(token)`: Debe verificar la validez del token y retornar un booleano o un error.\n\n3. **Herramientas**:\n   - Utilizar Jest 